### PYTHON CODE TO EXTRACT ID INFORMATION

I need to install the required library:

!pip install azure-ai-formrecognizer

I install azure-storage-blob to generate de SAS codes automatically in Python to access directly to the blob storage directly in step 5

!pip install azure-storage-blob

#### Import required Libraries

In [13]:
import os, json
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.storage.blob import (
    BlobSasPermissions,
    generate_blob_sas,
    BlobServiceClient
)
from datetime import datetime, timedelta

#### Azure authentication

In [ ]:
endpoint = "https://document-intelligence-udacity.cognitiveservices.azure.com/"
key      = "Add your key here"
client   = DocumentAnalysisClient(endpoint, AzureKeyCredential(key))
print(client)

#### Azure authentication Blob storage and SAS creation

In [ ]:
account_name = "001finalproject"
account_key = "Add your key here"
container_name = "step2"
blob_name = "digital_id_Alberto_Leon.pdf"

In [16]:
# SAS expliry time
sas_expiry = datetime.utcnow() + timedelta(hours=1)

# URL base
blob_url_base = f"https://{account_name}.blob.core.windows.net/{container_name}/{blob_name}"

# token generation
sas_token = generate_blob_sas(
    account_name=account_name,
    container_name=container_name,
    blob_name=blob_name,
    account_key=account_key,
    permission=BlobSasPermissions(read=True),
    expiry=sas_expiry
)

# Construir la URL completa
blob_url_with_sas = f"{blob_url_base}?{sas_token}"

In [17]:
print(blob_url_with_sas)

https://001finalproject.blob.core.windows.net/step2/digital_id_Alberto_Leon.pdf?se=2025-06-23T22%3A01%3A54Z&sp=r&sv=2025-05-05&sr=b&sig=Pke4ZtApI1AHsdYxWcgAHD0FI3f/%2B0oB/C6yPM9a/SA%3D


#### Analysis

In [19]:
# 1) Analyze the document 
poller  = client.begin_analyze_document_from_url(
            model_id="prebuilt-idDocument",   
            document_url=blob_url_with_sas,
            locale="en-US"                    
          )

result = poller.result()

# 2) Print the results
for doc in result.documents:
    fields = doc.fields
    data = {
        "FirstName":   fields.get("FirstName",   {}).value if "FirstName"   in fields else None,
        "LastName":    fields.get("LastName",    {}).value if "LastName"    in fields else None,
        "DateOfBirth": fields.get("DateOfBirth", {}).value if "DateOfBirth" in fields else None,
        "DocumentNum": fields.get("DocumentNumber", {}).value if "DocumentNumber" in fields else None,
    }
    print(json.dumps(data, indent=2, ensure_ascii=False))


{
  "FirstName": "Alberto",
  "LastName": "León",
  "DateOfBirth": null,
  "DocumentNum": "D4587987"
}
